# A simple knapsack

$$
\min~ cx \quad {s.t.}\quad wx \leq W \quad 0\leq x\leq 1

$$

The projected gradient descent updates:

- $x_{k+1} = proj_{C}(x_k - \eta \nabla f(x_k))$
- $f(x) = cx$
- $\nabla f(x) = c$
- $C$ is the feasible set

In [25]:
# import sys
# sys.path.insert(0, 'E:\\User\\Stevens\\Code\\Fold-opt\\fold_opt')

In [26]:
import torch
from torch import nn
import math
import cvxpy as cp

In [27]:
from GMRES import *
from fold_opt import *

# PGD

In [28]:
def proj_knapsack_closed(v, c, Q, max_iter: int = 25):
    """
    Water-filling projection (Duchi et al., 2008) – vectorised & differentiable
    v : (B,n)   raw iterate
    c : (n,)    positive costs   (Torch tensor)
    Q : float   budget
    """
    B, n = v.shape
    c = c.to(v)

    # 1) clip to orthant
    y     = v.clamp_min_(0.)
    cost  = (y * c).sum(1)
    mask  = cost > Q
    if not mask.any():
        return y

    # 2) batched bisection for λ s.t. Σ_i max(0, y_i − λ c_i)c_i = Q
    lam_lo = torch.zeros_like(cost)
    lam_hi = (y / c).max(1).values          # tight upper bound
    for _ in range(max_iter):
        lam   = 0.5 * (lam_lo + lam_hi)
        d_tmp = (y - lam.unsqueeze(1)*c).clamp_min_(0.)
        excess = (d_tmp*c).sum(1) - Q
        lam_lo = torch.where(excess > 0, lam, lam_lo)
        lam_hi = torch.where(excess <= 0, lam, lam_hi)

    lam  = lam_hi.unsqueeze(1)
    d_pf = (y - lam*c).clamp_min_(0.)
    return torch.where(mask.unsqueeze(1), d_pf, y)

In [29]:
def proj_knapsack_solver(v, c, Q):
    out = []
    c_np = c.cpu().numpy()
    for row in v.cpu().numpy():
        n   = row.size
        d   = cp.Variable(n)
        prob = cp.Problem(cp.Minimize(0.5*cp.sum_squares(d - row)),
                          [d >= 0, c_np @ d <= Q])
        prob.solve(solver=cp.OSQP, eps_abs=1e-8, verbose=False)
        out.append(torch.tensor(d.value, dtype=v.dtype))
    return torch.stack(out).to(v)

In [30]:
class PGDStep(torch.nn.Module):
    """
    update_step(c, d)   where   c == r   (risk/parameter vector).
    All constants are stored as buffers so autograd only tracks r.
    """
    def __init__(self, g, c_cost, Q: float, alpha: float,
                 lr: float = 5e-2, closed_proj: bool = True):
        super().__init__()
        self.register_buffer("g",      g)        # shape (n,)
        self.register_buffer("c_cost", c_cost)   # shape (n,)
        self.Q      = float(Q)
        self.alpha  = float(alpha)
        self.lr     = float(lr)
        self.closed = closed_proj

    def forward(self, r, d):
        """Projected-gradient step  d_{t+1} ← Π_Ω(d_t − lr ∇f)"""
        util = r * self.g
        if self.alpha == 1.0:
            grad = -(util / d.clamp_min(1e-12))
        else:
            grad = - (util**(1 - self.alpha)) * torch.pow(
                     d.clamp_min(1e-12), -self.alpha)

        d_new = d - self.lr * grad
        proj  = proj_knapsack_closed if self.closed else proj_knapsack_solver
        return proj(d_new, self.c_cost, self.Q)

In [31]:
def d_star_closed(r, g, c, alpha: float, Q: float):
    util = r * g
    if alpha == 0:
        idx = (util / c).argmax()
        d = torch.zeros_like(r); d[idx] = Q / c[idx]; return d
    if alpha == 1:
        return (Q / c.sum()) * torch.ones_like(r) / c
    if math.isinf(alpha):
        S = (c*c / util).sum()
        return Q * c / (util * S)
    num = torch.pow(c, -1/alpha) * torch.pow(util, 1/alpha - 1)
    return Q * num / num.sum()

def grad_d_star_closed(r, g, c, alpha: float, Q: float):
    d = d_star_closed(r, g, c, alpha, Q)
    term = (1/alpha - 1) * g / r
    G    = - torch.outer(d, d*term) / Q
    diag = d * term * (1 - d/Q)
    G[range(len(r)), range(len(r))] = diag
    return G            # shape (n,n)

In [32]:
def make_closed_form_solver(g, c_cost, Q: float, alpha: float):
    def solver(r_batch: torch.Tensor):
        return torch.stack([d_star_closed(r_i, g, c_cost, alpha, Q)
                            for r_i in r_batch], 0)
    return solver

In [33]:
# ----------------------------------------------------------
def my_solver(c):
    return torch.clamp(c, min=0.0)

def my_update_step(c, x):
    alpha = 0.1
    grad  = x - c
    x_new = torch.clamp(x - alpha*grad, min=0.0)
    return x_new

fold_layer = FoldOptLayer(
                solver      = my_solver,
                update_step = my_update_step,
                n_iter      = 20,
                backprop_rule='FPI')

c      = torch.tensor([[-1.0], [ 2.0]], requires_grad=True)   # (B=2, n=1)
target = torch.tensor([[ 1.0], [ 1.5]])

x_star = fold_layer(c)
print("x* from FoldOptLayer:", x_star.squeeze().tolist())     # → [0.0, 2.0]

loss = 0.5 * torch.sum((x_star - target) ** 2)
loss.backward()

print("Grad wrt c:", c.grad.squeeze().tolist())

x* from FoldOptLayer: [0.0, 2.0]
Grad wrt c: [0.0, 0.44468268752098083]


In [34]:
def proj_knapsack_closed(v, c, Q, nit=20):
    B, n = v.shape
    c = c.to(v)
    y = v.clamp_min_(0.)
    cost = (y*c).sum(1)
    infeas = cost > Q
    if not infeas.any():            # already feasible
        return y
    lam_lo = torch.zeros_like(cost)
    lam_hi = (y/c).max(1).values
    for _ in range(nit):
        lam = 0.5*(lam_lo+lam_hi)
        d   = (y - lam.unsqueeze(1)*c).clamp_min_(0.)
        excess = (d*c).sum(1) - Q
        lam_lo = torch.where(excess>0, lam, lam_lo)
        lam_hi = torch.where(excess<=0, lam, lam_hi)
    lam_final = lam_hi.unsqueeze(1)
    d_proj = (y - lam_final*c).clamp_min_(0.)
    return torch.where(infeas.unsqueeze(1), d_proj, y)

In [35]:
class PGDStep(torch.nn.Module):
    def __init__(self, g, c_cost, Q: float, alpha: float,
                 lr: float = 5e-2):
        super().__init__()
        self.register_buffer("g", g)
        self.register_buffer("c_cost", c_cost)
        self.Q     = float(Q)
        self.alpha = float(alpha)
        self.lr    = float(lr)

    def forward(self, r, d):
        util = r * self.g
        if self.alpha == 1.0:
            grad = -(util / d.clamp_min(1e-12))
        else:
            grad = -(util**(1-self.alpha)) * torch.pow(d.clamp_min(1e-12),
                                                      -self.alpha)
        d_new = d - self.lr*grad
        return proj_knapsack_closed(d_new, self.c_cost, self.Q)

In [36]:
# --- problem setup ----------------------------------------------------------
torch.manual_seed(0)
n      = 12
alpha  = 2.0
Q      = 7.0
g      = torch.rand(n) + 0.5
c_cost = torch.rand(n) + 0.5
r      = torch.rand(n, requires_grad=True) + 0.5

# --- components -------------------------------------------------------------
solver_opt = make_closed_form_solver(g, c_cost, Q, alpha)
pgd_step   = PGDStep(g, c_cost, Q, alpha, lr=4e-2)

layer = FoldOptLayer(
    solver      = solver_opt,
    update_step = pgd_step,
    n_iter      = 250,
    backprop_rule='GMRES')

# --- forward + backward -----------------------------------------------------
r_batched = r.unsqueeze(0)             # shape (1,n)
t0 = time.time()
d_out = layer(r_batched)               # (1,n), should equal d_star
loss  = d_out.sum()

# use autograd.grad to get ∂loss/∂r_batched
grad_r_batched = torch.autograd.grad(loss, r_batched)[0]
elapsed = (time.time() - t0)*1e3

# --- closed-form checks ----------------------------------------------------
with torch.no_grad():
    d_star  = d_star_closed(r, g, c_cost, alpha, Q)
    G_star  = grad_d_star_closed(r, g, c_cost, alpha, Q)

    # compare decisions
    dec_err  = torch.norm(d_out.squeeze(0) - d_star).item()
    # compare gradients
    grad_r   = grad_r_batched.squeeze(0)
    grad_err = (grad_r.unsqueeze(1) - G_star).abs().max().item()

print(f"‖d_out − d_star‖₂      = {dec_err:8.2e}")
print(f"max|∂d/∂r − closed|   = {grad_err:8.2e}")
print(f"forward+backward time = {elapsed:6.1f} ms")

‖d_out − d_star‖₂      = 0.00e+00
max|∂d/∂r − closed|   = 1.12e+00
forward+backward time =   19.0 ms


In [37]:
grad_r.unsqueeze(1)

tensor([[ 0.7764],
        [-0.2442],
        [-0.2030],
        [-0.7714],
        [-0.4356],
        [-0.2452],
        [-0.4158],
        [-0.1357],
        [-0.4115],
        [-0.2842],
        [-0.5305],
        [-0.5630]])